# RNN with MNIST

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

from torch.nn.utils import clip_grad_norm_

### Settings

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

seq_length = 28
input_size = 28
hidden_size = 128
batch_size = 100

num_epochs = 20
num_layers = 2
num_classes = 10

learning_rate = 0.01

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                         train=False,
                                         transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

### Model

In [ ]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        
        out = self.fc(out[:, -1, :])
        return out

### Loss & Optimizer

In [ ]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)

### Train

In [ ]:
total_step = len(train_loader)
curr_lr = learning_rate
loss_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        if(images.dim() == 4):
            images = images.reshape(-1, seq_length, input_size).to(device)
        else:
            images = images.to(device)
            
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                 .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            loss_list.append(loss.item())

### Result

In [ ]:
# 그래프 그리기==========
x = np.arange(len(loss_list))
plt.style.use(['ggplot'])
plt.plot(x, loss_list, marker='o', label='loss', markevery=100)
plt.xlabel("train step")
plt.ylabel("loss")
plt.ylim(0, 0.7)
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, seq_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 
